# Set-covering using Genetic Algorithm


In [17]:
import logging
import random
# from matplotlib import pyplot as plt
from functools import reduce
import math


In [18]:
logging.getLogger().setLevel(logging.DEBUG)

In [19]:
N = 500
POPULATION_SIZE = 2*N
NUM_GENERATIONS = 200
OFFSPRING_SIZE = math.ceil(1.5*N)

### Problem definition


In [20]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]


### Genetic operators and definition

In [21]:

class Individual:
    def __init__(self, problem, genome=None):
        if genome is None:
            self.genome = tuple([0]*len(problem))
        else:
            self.genome = genome
        self.problem = problem
        self.fitness = self.compute_fitness()        

    def __str__(self):
        return str([x for gene, x in zip(self.genome, self.problem) if gene == 1])

    def is_goal(self):
        return self.covered() == N

    def weight(self):
        return sum((len(el) for gene, el in zip(self.genome, self.problem) if gene == 1))
    
    def covered(self):
        sol = (element for gene, element in zip(self.genome, self.problem) if gene == 1)
        return len( reduce(lambda a, b: set(a) | set(b), sol, set()) )

    def compute_fitness(self):
        # covered number / weight of the solution + bonus if it is a goal solution
        # return 0.3*self.covered() - 0.1*self.weight() #+ N*self.is_goal()
        return (self.covered(), -self.weight())

#### Genetic operators


In [22]:
def crossover(g1, g2):
    cut_point = random.randint(0, len(g1))
    return g1[:cut_point] + g2[cut_point:]

def mutation(g):
    mutation_point = random.randint(0, len(g) - 1)
    return g[:mutation_point] + (1 - g[mutation_point],) + g[mutation_point + 1:]

def tournament(population, tournament_size=2):
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)



## Genetic algorithm

### Initial Population


In [23]:
all_states = list(set([tuple(x) for x in problem(N, seed=42)]))

# logging.debug(f"All states: {all_states}")
#logging.debug(f"All states: {all_states_b}")

population = [Individual(all_states) for _ in range(POPULATION_SIZE)]

### Evolution

In [24]:
LOG_FREQUENCY = 10

for generation in range(NUM_GENERATIONS):
    offspring = list()
    for i in range(OFFSPRING_SIZE):
        if random.random() < 0.3:
            p = tournament(population)
            o = mutation(p.genome)
        else:
            p1 = tournament(population)
            p2 = tournament(population)
            o = crossover(p1.genome, p2.genome)
        offspring.append(Individual(all_states, o))
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
    best_one = population[0]
    if (generation) % LOG_FREQUENCY == 0:
        logging.debug(f"#{generation + 1} fitness={best_one.fitness}, w = {best_one.weight()}") 
    # if best_one.weight()/N < 1.5 and best_one.is_goal():
    #     break


DEBUG:root:#1 fitness=(201, -201), w = 201
DEBUG:root:#11 fitness=(500, -2066), w = 2066
DEBUG:root:#21 fitness=(500, -1762), w = 1762
DEBUG:root:#31 fitness=(500, -1632), w = 1632
DEBUG:root:#41 fitness=(500, -1562), w = 1562
DEBUG:root:#51 fitness=(500, -1557), w = 1557
DEBUG:root:#61 fitness=(500, -1557), w = 1557
DEBUG:root:#71 fitness=(500, -1557), w = 1557
DEBUG:root:#81 fitness=(500, -1557), w = 1557
DEBUG:root:#91 fitness=(500, -1557), w = 1557
DEBUG:root:#101 fitness=(500, -1557), w = 1557
DEBUG:root:#111 fitness=(500, -1557), w = 1557


KeyboardInterrupt: 

In [25]:
print(f"Number of Generation: {generation+1}")
print(f"#{N} : weight: {population[0].weight()}, fitness = {population[0].fitness}, sol={population[0].is_goal()}")

Number of Generation: 118
#500 : weight: 1557, fitness = (500, -1557), sol=True
